# 비화재보 분류 


---
## 1. 데이터 전처리     
---

### 각 년도별 데이터 결합 

In [1]:
import pandas as pd
import numpy as np 
import re
import pickle
import xlrd

start_year= 2016
end_year =2021

In [2]:
df_all =pd.DataFrame()
#년도합치기
for year in range(start_year,end_year+1):
#시트 이름 가져오기    
    wb = xlrd.open_workbook(f"{year}년 전체 신고건수_관할 및 재난주소 추가.xls")
    globals()[f'df_{year}']=pd.DataFrame()
#Sheet 합치기
    for  i in range(len(wb.sheets())):
        tmp = pd.read_excel(f"{year}년 전체 신고건수_관할 및 재난주소 추가.xls",sheet_name = wb.sheet_names()[i])['신고내용'].dropna(axis=0).T
        globals()[f'df_{year}']= pd.concat([globals()[f'df_{year}'],tmp], axis = 0)
    df_all= pd.concat([df_all,globals()[f'df_{year}']], axis = 0)

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

##### 임시저장

In [13]:
# with open(f"119_신고내용_{start_year} - {end_year}.pickle","wb") as fw:
#     pickle.dump(df_all, fw)   

In [2]:
with open(f"119_신고내용_{start_year} - {end_year}.pickle",'rb') as fr:
    df_all = pickle.load(fr)

### 중복 문자열 제거

In [3]:
print(len(df_all))
df_all= pd.DataFrame(set(df_all[0]))
print(len(df_all))
print(df_all.head(3))

1711724
1368100
                                  0
0  [의식 있음] 내당홈플러스 3층 꽃집 앞 / 여성 쓰러짐 
1        동구시장 풀과 나무 한의원//호흡곤란 사지 마비
2                ★★주간 기동순찰 및 훈련출동★★


###  신고내용 전처리 

In [4]:
from konlpy.tag import Okt 
okt = Okt()
print(okt.pos('속보기'))  #속보기 한 단어로 인식 X   -> AAA로 임시 치환
print(okt.pos('오작동')) # 오+작동으로 분리된 후 오가 제거되어 의미가 왜곡 -> BBB로 임시 치환
# 문제X
print(okt.pos('경보기'),okt.pos('경보'),okt.pos('속보')) 

[('속', 'Modifier'), ('보기', 'Noun')]
[('오', 'Modifier'), ('작동', 'Noun')]
[('경보기', 'Noun')] [('경보', 'Noun')] [('속보', 'Noun')]


In [5]:
df_all2 = df_all.copy()
for i in range(len(df_all2)):
    tmp = re.sub(r'[^가-힣+ ]',' ',df_all2.iloc[i,0]) # 한글말고 제거
    tmp = re.sub('속보기','AAA',tmp) # 속보기 ->A로 단어 치환    
    tmp = re.sub('오작동','BBB',tmp) # 오작동 ->B로 단어 치환    
    df_all2.iloc[i,0] = tmp
df_all2.iloc[702307 ,0]=re.sub('속보기','속보', df_all2.iloc[702307 ,0]) # 알수없는 이유로 치환이 안됨
print(df_all2.head(3))
print(len(df_all2))

                                  0
0   의식 있음  내당홈플러스  층 꽃집 앞   여성 쓰러짐 
1        동구시장 풀과 나무 한의원  호흡곤란 사지 마비
2                  주간 기동순찰 및 훈련출동  
1368100


In [6]:
#속보기 단어 제거 확인
print(f'속보기 단어 변경 전   : {df_all[0].apply(lambda x: re.search("속보기", x)is not None).sum()}')
print(f'속보기 단어 변경 후 :{df_all2[0].apply(lambda x: re.search("속보기", x)is not None).sum()}')
print(f'속보기 단어 변경 전   : {df_all[0].apply(lambda x: re.search("오작동", x)is not None).sum()}')
print(f'오작동 단어 변경 후 :{df_all2[0].apply(lambda x: re.search("오작동", x)is not None).sum()}')
print(f'AAA 단어 변경 후 :{df_all2[0].apply(lambda x: re.search("AAA", x)is not None).sum()}')
print(f'BBB 단어 변경 후 :{df_all2[0].apply(lambda x: re.search("BBB", x)is not None).sum()}')
# print(f'제거 안된 문장 :{df_all2[0][df_all2[0].apply(lambda x: re.search("속보기", x)is not None)]}')

속보기 단어 변경 전   : 7937
속보기 단어 변경 후 :0
속보기 단어 변경 전   : 9987
오작동 단어 변경 후 :0
AAA 단어 변경 후 :7937
BBB 단어 변경 후 :9987


###  문장 -> 형태소 형태로 변환
형태소 분석(Noun, Adjective) -명사와 형용사만  
1 글자를 배제  -의미가 거의X  
불용어 제외  

In [7]:

sents = []
for i in range(len(df_all2[0])):
    print("\r[tokenizing]  {}  /  {}   {}  %".format(i+1, len(df_all2[0]), round( (i / len(df_all2[0])) * 100 , 1 ) ), end = '    ', flush = False)
    pos_res = okt.pos(df_all2[0].iloc[i])
    
    STOP_WORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    keywords = []
    for word, pos in pos_res:
        if ((pos == "Noun")|(pos == "Adjective")| (pos =='Alpha'))&(len(word) >= 2)&(word not in STOP_WORDS):
            word.replace('AAA', '속보기').replace('BBB','오작동') # 원래 단어로 되돌리기
            keywords.append(word)
    sents.append(' '.join(keywords))
# print(sents[0:5])

[tokenizing]  1  /  1368100   0.0  %    

###  전처리 전후 키워드 갯수 확인
비화재보 주요 키워드 : 경보, 속보  
비화재보 제외 키워드 : 테스트, 오인주의, 훈련, 점검   

#### 경보, 속보  

In [ ]:
check = []
for i in range(len(sents)) :
    if '속보' in sents[i]:
            check.append(sents[i])
print(f'기존 [속보] 단어 수: {df_all2[0].apply(lambda x: re.search("속보", x)is not None).sum()}')
print(f'전처리 후 [속보] 단어 수: {len(check)}\n')  
check = []
for i in range(len(sents)) :
    if '경보' in sents[i]:
            check.append(sents[i])
print(f'기존 [경보] 단어 수: {df_all2[0].apply(lambda x: re.search("경보", x)is not None).sum()}')
print(f'전처리 후 [경보] 단어 수: {len(check)}\n')  

기존 [속보] 단어 수: 16048
전처리 후 [속보] 단어 수: 16038


#### 테스트, 오인주의, 훈련, 점검   

In [ ]:
check = []
for i in range(len(sents)) :
    if '점검' in sents[i]:
            check.append(sents[i])
print(f'기존 [점검] 단어 수: {df_all2[0].apply(lambda x: re.search("점검", x)is not None).sum()}')
print(f'전처리 후 [점검] 단어 수: {len(check)}\n')  
# df_all2[df_all2[0].apply(lambda x: re.search("점검", x)is not None)]
check = []
for i in range(len(sents)) :
    if '훈련' in sents[i]:
            check.append(sents[i])
print(f'기존 [훈련] 단어 수: {df_all2[0].apply(lambda x: re.search("훈련", x)is not None).sum()}')
print(f'전처리 후 [훈련] 단어 수: {len(check)}\n')  
# df_all2[df_all2[0].apply(lambda x: re.search("훈련", x)is not None)]
check = []
for i in range(len(sents)) :
    if '오인' in sents[i]:
            check.append(sents[i])
print(f'기존 [오인] 단어 수: {df_all2[0].apply(lambda x: re.search("오인", x)is not None).sum()}')
print(f'전처리 후 [오인] 단어 수: {len(check)}\n')  
# df_all2[df_all2[0].apply(lambda x: re.search("오인", x)is not None)]
check = []
for i in range(len(sents)) :
    if '테스트' in sents[i]:
            check.append(sents[i])
print(f'기존 [테스트] 단어 수: {df_all2[0].apply(lambda x: re.search("테스트", x)is not None).sum()}')
print(f'전처리 후 [테스트] 단어 수: {len(check)}\n')  
# df_all2[df_all2[0].apply(lambda x: re.search("테스트", x)is not None)]
check = []
for i in range(len(sents)) :
    if 'A' in sents[i]:
            check.append(sents[i])
print(f'기존 [A] 단어 수: {df_all2[0].apply(lambda x: re.search("A", x)is not None).sum()}')
print(f'전처리 후 [A] 단어 수: {len(check)}\n')  
check = []
for i in range(len(sents)) :
    if 'B' in sents[i]:
            check.append(sents[i])
print(f'기존 [B] 단어 수: {df_all2[0].apply(lambda x: re.search("B", x)is not None).sum()}')
print(f'전처리 후 [B] 단어 수: {len(check)}\n')  
check = []
for i in range(len(sents)) :
    if '속보기' in sents[i]:
            check.append(sents[i])
print(f'기존 [속보기] 단어 수: {df_all2[0].apply(lambda x: re.search("속보기", x)is not None).sum()}')
print(f'전처리 후 [속보기] 단어 수: {len(check)}\n')  
check = []
for i in range(len(sents)) :
    if '오작동' in sents[i]:
            check.append(sents[i])
print(f'기존 [오작동] 단어 수: {df_all2[0].apply(lambda x: re.search("오작동", x)is not None).sum()}')
print(f'전처리 후 [오작동] 단어 수: {len(check)}\n')  

기존 [오인] 단어 수: 12399
전처리 후 [오인] 단어 수: 12188


,0
63,상인 동 동사무소 뒷쪽 연막소독 실시 오인신고 주의 분정도 소요
78,오인주의 기쁜소식 대구교회 밤 시까지 화목보일러 덴다 추가 월 일...
467,안심중학교 맞은편 낙엽 소각 오인 주의
507,방역소독 오인주의 내일교회 주변일대 시 분까지
603,오인주의 방역 경상감영공원 시 분까지
...,...
1367714,농작물 소각에 대한 신고 소각행위는 지정된 장소에서 허가받아 가능하며 에서는...
1367723,보일러 가동 연기 발생 오인신고 주의
1367851,오인주의 용산동 모닝빌라 용산네거리 동편 까지 방역소독
1367887,북구보건소 방역팀 경대교 도청교 성북교 방향 하천쪽에서 연무소독 분정도 ...


In [ ]:
check = []
for i in range(len(sents)) :
    if 'A' in sents[i]:
            check.append(sents[i])
print(f'기존 [A] 단어 수: {df_all2[0].apply(lambda x: re.search("A", x)is not None).sum()}')
print(f'전처리 후 [A] 단어 수: {len(check)}')  
check = []
for i in range(len(sents)) :
    if 'B' in sents[i]:
            check.append(sents[i])
print(f'기존 [B] 단어 수: {df_all2[0].apply(lambda x: re.search("B", x)is not None).sum()}')
print(f'전처리 후 [B] 단어 수: {len(check)}')  
check = []
for i in range(len(sents)) :
    if '속보기' in sents[i]:
            check.append(sents[i])
print(f'기존 [속보기] 단어 수: {df_all2[0].apply(lambda x: re.search("속보기", x)is not None).sum()}')
print(f'전처리 후 [속보기] 단어 수: {len(check)}')  
check = []
for i in range(len(sents)) :
    if '오작동' in sents[i]:
            check.append(sents[i])
print(f'기존 [오작동] 단어 수: {df_all2[0].apply(lambda x: re.search("오작동", x)is not None).sum()}')
print(f'전처리 후 [오작동] 단어 수: {len(check)}')  

##### 임시저장

In [ ]:
with open(f"119전처리_{start_year}-{end_year}.pickle","wb") as fw:
    pickle.dump(sents, fw)   

In [22]:
# with open(f"119전처리_{start_year}-{end_year}.pickle",'rb') as fr:
#     sents = pickle.load(fr)

#### 비화재보 /화재 데이터 분리

In [521]:
#화재보
df_sents = pd.DataFrame(sents)
non_fire=[]
for i in range(len(sents)):
    if (('경보' in df_sents.iloc[i,0])|('속보' in df_sents.iloc[i,0])) and (('점검' not in df_sents.iloc[i,0])&('훈련' not in df_sents.iloc[i,0])&('오인' not in df_sents.iloc[i,0])&('테스트' not in df_sents.iloc[i,0])):
        non_fire.append(df_sents.iloc[i,0])
print( f'비화재보 / 전체 데이터 : {len(non_fire)}/{len(sents)} \n비화재보 비율 : {round((len(non_fire)/ len(sents))*100,2)}%')        

비화재보 / 전체 데이터 : 501/27432 
비화재보 비율 : 1.83%


In [442]:
df_non_fire=pd.DataFrame(non_fire)
print(f'[경보] 단어 수: {df_non_fire[0].apply(lambda x: re.search("경보", x)is not None).sum()}')
print(f'[속보] 단어 수: {df_non_fire[0].apply(lambda x: re.search("속보", x)is not None).sum()}')
print(f'[테스트] 단어 수: {df_non_fire[0].apply(lambda x: re.search("테스트", x)is not None).sum()}')
print(f'[훈련] 단어 수: {df_non_fire[0].apply(lambda x: re.search("훈련", x)is not None).sum()}')
print(f'[오인] 단어 수: {df_non_fire[0].apply(lambda x: re.search("오인", x)is not None).sum()}')
print(f'[점검] 단어 수: {df_non_fire[0].apply(lambda x: re.search("점검", x)is not None).sum()}')
# df_non_fire[df_non_fire[0].apply(lambda x: re.search("테스트", x)is not None)]

[경보] 단어 수: 15571
[속보] 단어 수: 9942
[테스트] 단어 수: 0
[훈련] 단어 수: 0
[오인] 단어 수: 0
[점검] 단어 수: 0


---
## 2. 분석
---

### CountVectorizer Vectorizing


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
cv_sents=vect.fit_transform(sents)
print(cv_sents.shape)
# cv_data=cv_sents.toarray()
# cv_data

(1368100, 69363)


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(2)  
pca.fit(cv_sents)
X_pca = pca.transform(cv_sents)
print(X_pca.shape)

### TF-IDF Vectorizing


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf_sents =vectorizer.fit_transform(sents)
print(tfidf_sents.shape)

(1368100, 69363)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.0, analyzer='char', sublinear_tf=True, ngram_range=(1,3), max_features=10000)
tfidf_sents =vectorizer.fit_transform(sents)
print(tfidf_sents.shape)
# 파라미터에 값입력

(778936, 10000)


### Word2vec(CBOW)


#### 전체데이터

In [38]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []
for sen in sents:
    sentences.append(sen.split())

In [39]:
# 파라미터
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3 # 0.001

In [ ]:
from gensim.models import word2vec
print("Training model ....")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count=min_word_count, window=context, sample=downsampling)

In [67]:
tmp1 = model.wv.most_similar("속보")
tmp2= model.wv.most_similar("경보")
print(f'속보\n: {tmp1} \n')
print(f'경보\n: {tmp2} \n')

속보
: [('설비', 0.5565919876098633), ('자동', 0.5560749769210815), ('경보', 0.5363364815711975), ('소방설비', 0.5336592197418213), ('작동', 0.4983700215816498), ('인과', 0.49686312675476074), ('소방시설', 0.4892948269844055), ('점검', 0.48302412033081055), ('경보기', 0.47331950068473816), ('테스트', 0.4692937731742859)] 

경보
: [('화재경보기', 0.7432859539985657), ('보가', 0.7132909297943115), ('경보기', 0.7123016715049744), ('경종', 0.6195616722106934), ('감지기', 0.6176596879959106), ('수신기', 0.603028416633606), ('알람', 0.5819022059440613), ('번쩍', 0.5372390151023865), ('속보', 0.5363364815711975), ('발신기', 0.534061074256897)] 



In [41]:
def get_features(words,model, num_features):
    feature_vector = np.zeros((num_features), dtype=np.float32)
    num_words = 0
    index2word_set = set(model.wv.index2word)
    for w in words:
        if w in index2word_set:
            num_words +=1
            feature_vector = np.add(feature_vector, model[w])
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [42]:
def get_dataset(sentence, model, num_features):
    dataset = list()
    for s in sentence:
        dataset.append(get_features(s, model, num_features))
    FeatureVecs = np.stack(dataset)
    return FeatureVecs

In [43]:
train_data_vecs = get_dataset(sentences, model, num_features)
print(train_data_vecs.shape)

C:\Users\user\AppData\Local\Temp/ipykernel_2544/2435305589.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  feature_vector = np.add(feature_vector, model[w])
C:\Users\user\AppData\Local\Temp/ipykernel_2544/2435305589.py:9: RuntimeWarning: invalid value encountered in true_divide
  feature_vector = np.divide(feature_vector, num_words)


(1368100, 300)


#### 비화재보 데이터

In [458]:
nonfire_sentences = []
for sen in non_fire:
    nonfire_sentences.append(sen.split())

In [ ]:
from gensim.models import word2vec
print("Training model ....")
model_2 = word2vec.Word2Vec(nonfire_sentences, workers=num_workers, size=num_features, min_count=min_word_count, window=context, sample=downsampling)

In [461]:
tmp1 = model_2.wv.most_similar("속보")
tmp2= model_2.wv.most_similar("경보")
print(f'속보\n: {tmp1} \n')
print(f'경보\n: {tmp2} \n')

속보
: [('국가', 0.8422040939331055), ('단대', 0.8356016874313354), ('한국', 0.8290706276893616), ('산업', 0.8263710141181946), ('요양원', 0.8251696825027466), ('자동', 0.8218286037445068), ('양병', 0.8176388740539551), ('재활', 0.8150773048400879), ('테크', 0.8133792281150818), ('주식회사', 0.8119539618492126)] 

경보
: [('소리', 0.8460812568664551), ('계속', 0.8451510667800903), ('방송', 0.8240757584571838), ('일반', 0.8058801889419556), ('경로', 0.804393470287323), ('거주', 0.7888182997703552), ('인근', 0.7832477688789368), ('있다', 0.7658571004867554), ('있는', 0.7640323042869568), ('있는데', 0.7619271874427795)] 



In [479]:
word1='래미안'
word2='국민은행'
word3=''

check = df_all[df_all[0].apply(lambda x: re.search(word1, x)is not None)]
check = check[check[0].apply(lambda x: re.search(word2, x)is not None)]
check = check[check[0].apply(lambda x: re.search(word3, x)is not None)]
check


,0
846,성당래미안2단지와 3단지 사이 국민은행 맞은편 건물 경보기 오작동
